# Introduction

This notebook will train a very simple model that will compare two numbers and tell us if one is larger than the other.

We'll convert the model into something that can be used by tflite and then run it on the ESP32

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.data import Dataset
import numpy as np
import itertools

## Create a dataset to train our model
We'll create a python generator and feed that through a tensorflow Dataset to train our model

In [26]:
def data_generator():
    while(True):
        number1 = np.random.uniform();
        number2 = np.random.uniform();
        # our input data is an array containing 2 numbers
        X = [number1, number2]
        # our label is 1 or 0
        Y = 1 if number2 > number1 else 0
        # our generator should return the input data and the label
        yield X, [Y]

# create a dataset from our generator
train_dataset = tf.data.Dataset.from_generator(
    data_generator,
    output_types = (tf.float32, tf.int32),
    output_shapes=((2), (1))
)
train_dataset = train_dataset.batch(batch_size=30)

## Our very simple mode

We don't need a very complicated model for our problem, so we'll just define a small neural network with an input layer and an output layer.

It's important that the activation function for the output should be sigmoid. This activation function will output a value between 0 and 1.

In [27]:
model = Sequential([
    Input(shape=(2)),
    Dense(10, activation='relu'),
    Dense(50, activation='relu'),
    Dense(150, activation='relu'),
    Dense(500, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

## Compile our model¶
For our loss function we need to use BinaryCrossentropy.

Crossentropy quantifies the difference between two probability distribution.

We have a binary distribution (True or False) so we use binary crossentropy to compare the output from our model with the true distribution.


In [28]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 10)                30        
                                                                 
 dense_31 (Dense)            (None, 50)                550       
                                                                 
 dense_32 (Dense)            (None, 150)               7650      
                                                                 
 dense_33 (Dense)            (None, 500)               75500     
                                                                 
 dense_34 (Dense)            (None, 10)                5010      
                                                                 
 dense_35 (Dense)            (None, 1)                 11        
                                                                 
Total params: 88,751
Trainable params: 88,751
Non-trai

In [29]:
model.fit(
    train_dataset,
    steps_per_epoch=50,
    epochs=4
)

Epoch 1/4
50/50 [==============================] - 1s 7ms/step - loss: 0.5183 - accuracy: 0.7060
Epoch 2/4
50/50 [==============================] - 0s 7ms/step - loss: 0.0632 - accuracy: 0.9933
Epoch 3/4
50/50 [==============================] - 0s 7ms/step - loss: 0.0430 - accuracy: 0.9833
Epoch 4/4
50/50 [==============================] - 0s 7ms/step - loss: 0.0326 - accuracy: 0.9847


## Testing our model
We can feed in some values and see what our model predicts

In [30]:
test_X = np.array([
    [0.1, 0.2],
    [0.3, 0.4],
    [0.5, 0.1],
    [0.7, 0.2]
])
Y = model.predict_on_batch(test_X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(Y)

[[0.99]
 [1.00]
 [0.00]
 [0.00]]


## Export our model for tflite
We need to convert our model into a tflite model

In [31]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset_gen():
    for _ in range(10000):
        yield [
            np.array(
                [np.random.uniform(), np.random.uniform()]
            , dtype=np.float32)
        ]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


95360

## To convert to C++
We can then run this command to convert the model to c code.
```
xxd -i converted_model.tflite > mdata.cc
```

In [32]:
!xxd -i converted_model.tflite > mdata.cc